In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%%capture
pip install transformers

In [ ]:
import os
import re
import torch 
import torch.nn as nn
import torch.nn.functional as F
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.optim import Adam
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import BertTokenizerFast, BertForTokenClassification, BertTokenizer, BertConfig, GPT2Tokenizer, GPT2TokenizerFast
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader

In [ ]:
ROOT_DIR = "/content/gdrive/MyDrive/ResumeRanker"

In [ ]:
def extract_data_from_json(filepath):
    text_dataset = []
    dataset = []
    with open(filepath, 'r') as f:
        lines = f.readlines()

    for line in tqdm(lines,desc='Extracting Data    '):
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        data_annotations = data['annotation']
        entities = []
        if data_annotations is not None:
            for annotation in data_annotations:
                point = annotation['points'][0]
                labels = annotation['label']
                if isinstance(labels, list):
                    if not labels:
                        continue
                    label = labels[0]
                else:
                    label = labels

                point_start = point['start']
                point_end = point['end']
                point_text = point['text']
                
                lspace = len(point_text) - len(point_text.lstrip())
                rspace = len(point_text) - len(point_text.rstrip())
                if lspace != 0:
                    point_start = point_start + lspace
                if rspace != 0:
                    point_end = point_end - rspace
                entities.append((point_start, point_end + 1 , label))
        dataset.append((text, {"entities" : entities}))
        text_dataset.append(text)

    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in tqdm(dataset,desc='Processing Entities'):
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append((text, {'entities': valid_entities}))

    labels = []
    s = 0
    for i in tqdm(range(len(cleaned_data)),desc='Creating Labels    '):
        start = 0
        emptyList = ["Empty"] * len(cleaned_data[i][0].split())
        numberOfWords = 0
        lenOfString = len(cleaned_data[i][0])
        strData = cleaned_data[i][0]
        strDictData = cleaned_data[i][1]
        lastIndexOfSpace = strData.rfind(' ')
        for i in range(lenOfString):
            if (strData[i]==" " and strData[i+1]!=" "):
                for k,v in strDictData.items():
                    for j in range(len(v)):
                        entList = v[len(v)-j-1]
                        if (start>=int(entList[0]) and i<=int(entList[1])):
                            emptyList[numberOfWords] = entList[2]
                            break
                        else:
                            continue
                start = i + 1  
                numberOfWords += 1
            if (i == lastIndexOfSpace):
                for j in range(len(v)):
                        entList = v[len(v)-j-1]
                        if (lastIndexOfSpace>=int(entList[0]) and lenOfString<=int(entList[1])):
                            emptyList[numberOfWords] = entList[2]
                            numberOfWords += 1
        labels.append(emptyList)
        s = s + numberOfWords
    return text_dataset, labels

In [ ]:
json_file_path = "/content/gdrive/MyDrive/ResumeRanker/Dataset/Entity Recognition in Resumes.json"
text, labels = extract_data_from_json(json_file_path)

Creating Labels    : 100%|██████████| 220/220 [00:00<00:00, 235.83it/s]


In [ ]:
tags_id = {"Name":0,"College Name":1,"Degree":2,"Graduation Year":3,"Years of Experience":4,"Companies worked at":5,"Designation":6,"Skills":7,"Location":8,"Email Address":9,"UNKNOWN":10,"Empty":11}

In [ ]:
total_labels = []
for i in labels:
  total_labels.extend(i)

from collections import Counter
dist = Counter(total_labels)
weights = [1/dist[i] for i in tags_id]
sum_ = sum(weights)
weights = [i/sum_ for i in weights]
print(weights)
print(sum(weights))

[0.008464584949229933, 0.0038598507368488495, 0.004701401628317722, 0.05760971249028134, 0.050787509695379596, 0.004396185349486161, 0.0037730701239969203, 0.0005190762152835999, 0.042415942163174174, 0.051464676491318, 0.77197014736977, 3.784278691381952e-05]
1.0000000000000002


In [ ]:
class NERconfig:
    def __init__(self, maxlen, batch_size, rootdir, tag_id, pretrained = True, use_scheduler = False):
        self.maxlen = maxlen
        self.batch_size = batch_size
        self.rootdir = rootdir
        self.tag_id = tag_id
        self.id_tag = dict((v,k) for k,v in self.tag_id.items())
        self.pretrained = pretrained
        self.use_scheduler = use_scheduler
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class NER_Dataset(Dataset):
    def __init__(self, text_data, entity_labels, config,  tokenizer):
        self.text_data = text_data
        self.entity_labels = entity_labels
        self.tokenizer = tokenizer
        self.config = config
        self.text = [self.tokenizer(i, add_special_tokens=False, return_tensors="pt") for i in self.text_data]
        self.labels = [self.align_label(i,j) for i,j in zip(self.text_data, self.entity_labels)]
        self.X = []
        self.Y = []
        self.restructure_data()
    
    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        return x, y

    def restructure_data(self):
        for i in range(len(self.text)):
            input_id_chunks = self.text[i]['input_ids'][0].split(self.config.maxlen-2)
            mask_chunks = self.text[i]['attention_mask'][0].split(self.config.maxlen-2)
            label_chunks = torch.Tensor(self.labels[i]).split(self.config.maxlen-2)
            for j in range(len(input_id_chunks)):
                inp_chunk = torch.cat([torch.Tensor([101]),input_id_chunks[j],torch.Tensor([102])])
                msk_chunk = torch.cat([torch.Tensor([1]), mask_chunks[j], torch.Tensor([1])])
                lab_chunk = torch.cat([torch.Tensor([self.config.tag_id["Empty"]]), label_chunks[j], torch.Tensor([self.config.tag_id["Empty"]])])
                pad_len = self.config.maxlen - inp_chunk.shape[0]
                if pad_len > 0:
                    inp_chunk = torch.cat([inp_chunk, torch.Tensor([0] * pad_len)])
                    msk_chunk = torch.cat([msk_chunk, torch.Tensor([0] * pad_len)])
                    lab_chunk = torch.cat([lab_chunk, torch.Tensor([self.config.tag_id["Empty"]] * pad_len)])
                input_dict = {
                              'input_ids': inp_chunk.long(),
                              'attention_mask': msk_chunk.int()
                              }
                self.X.append(input_dict)
                self.Y.append(lab_chunk.long())


    def align_label(self, texts, labels):
        split_tokens = texts.split()
        comp_idx = 0
        extension = 0
        tokenized_inputs = self.tokenizer(texts,add_special_tokens=False)
        word_ids = tokenized_inputs.word_ids()
        label_ids = []
        i = 0
        while i < len(word_ids):
            word = self.tokenizer.decode(tokenized_inputs.input_ids[i-extension:i+1])
            if (word == split_tokens[comp_idx]) or ("".join(word.split()) == split_tokens[comp_idx]):
                label_ids.extend([self.config.tag_id[labels[comp_idx]]]*(extension+1))
                comp_idx += 1
                extension = 0
            elif word == '[UNK]':
                label_ids.extend([self.config.tag_id[labels[comp_idx]]]*(extension+1))
                comp_idx += 1
                extension = 0
            else:
                extension += 1
            i += 1
        return label_ids

In [ ]:
class Trainer:
    def __init__(self, model, config, optimizer, tokenizer, loss_func, train_dataset, test_dataset,
                 val_dataset = None):

        self.model = model
        self.optimizer = optimizer
        self.config = config
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        self.train_dataloader = DataLoader(self.train_dataset, batch_size = self.config.batch_size, shuffle = True)
        self.valid_dataloader = DataLoader(self.val_dataset, batch_size = self.config.batch_size, shuffle = True)
        self.test_dataloader = DataLoader(self.test_dataset, batch_size = self.config.batch_size, shuffle = True)
        self.device = self.config.device
        self.criterion = loss_func.to(self.device)
        self.best_loss = float('inf')
        self.load_pretrained = self.config.pretrained
        self.use_scheduler = self.config.use_scheduler 
        if self.use_scheduler:
            self.scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=0.1, patience=3, verbose=True)

        if os.path.exists(os.path.join(self.config.rootdir,"Models","BERT_NER","BERTF_NER.pth")) and self.load_pretrained:
            print("Pretrained Weights found. Loading Pretrained weights...")
            self.load_weights(os.path.join(self.config.rootdir,"Models","BERT_NER","BERTF_NER.pth"))
        self.model.to(self.device)

    def compute_loss(self, logits, labels, attention_mask=None):
        # Only keep active parts of the loss
        if attention_mask is not None:
            active_loss = attention_mask.view(-1) == 1
            active_logits = logits.view(-1, len(config.tag_id))
            active_labels = torch.where(
                active_loss, labels.view(-1), torch.tensor(-100).type_as(labels)
            )
            loss = self.criterion(active_logits, active_labels)
        else:
            loss = self.criterion(logits.view(-1, self.num_labels), labels.view(-1))
        return loss*1000


    def train_one_epoch(self):
        train_loss = 0
        train_acc = 0
        loader1 = enumerate(tqdm(self.train_dataloader))
        for i,(X,Y) in loader1:
            train_label = Y.to(self.device)
            mask = X['attention_mask'].to(self.device)
            input_id = X['input_ids'].to(self.device)
            self.optimizer.zero_grad()
            output = self.model(input_id, mask)
            logits = output.logits
            loss = self.compute_loss(logits,train_label,mask)
            loss.backward()
            self.optimizer.step()
            predictions = logits.argmax(dim=2)
            acc = (predictions == train_label).float().mean()
            train_acc+=acc.item()
            train_loss+=loss.item()
        train_loss/=(i+1)
        train_acc/=(i+1)
        return train_loss,train_acc

    def valid_one_epoch(self):
        valid_loss = 0
        valid_acc = 0
        loader2 = enumerate(tqdm(self.valid_dataloader)) 
        self.model.eval()
        for i,(X,Y) in loader2:
            val_label = Y.to(self.device)
            mask = X['attention_mask'].to(self.device)
            input_id = X['input_ids'].to(self.device)
            output = self.model(input_id, mask)
            logits = output.logits
            loss = self.compute_loss(logits,val_label,mask)
            predictions = logits.argmax(dim=2)
            acc = (predictions == val_label).float().mean()
            valid_acc+=acc.item()
            valid_loss+=loss.item()
        valid_loss/=(i+1)
        valid_acc/=(i+1)
        self.model.train()
        return valid_loss, valid_acc

    def test(self):
        test_loss = 0
        test_acc = 0
        loader2 = enumerate(tqdm(self.valid_dataloader)) 
        self.model.eval()
        for i,(X,Y) in loader2:
            val_label = Y.to(self.device)
            mask = X['attention_mask'].to(self.device)
            input_id = X['input_ids'].to(self.device)
            output = self.model(input_id, mask)
            logits = output.logits
            loss = self.compute_loss(logits,val_label,mask)
            predictions = logits.argmax(dim=2)
            acc = (predictions == val_label).float().mean()
            test_acc+=acc.item()
            test_loss+=loss.item()
        test_loss/=(i+1)
        test_acc/=(i+1)
        self.model.train()
        return test_loss, test_acc

    def fit(self, num_epochs):
        train_losses = []
        train_accuracies = []
        valid_losses = []
        valid_accuracies = []
        for epoch in range(num_epochs):
            train_loss, train_acc = self.train_one_epoch()
            if self.valid_dataloader is None:
                valid_loss = 0
                valid_acc = 0
            else:
                valid_loss, valid_acc = self.valid_one_epoch()
            train_losses.append(train_loss)
            train_accuracies.append(train_acc)
            valid_losses.append(valid_loss)
            valid_accuracies.append(valid_acc)
            if self.valid_dataloader is None:
                print("epoch {} | train loss : {} | train acc : {}".format(epoch, train_loss, train_acc))
            else:
                print("epoch {} | train loss : {} | train acc : {} | valid loss : {} | valid acc : {}".format(epoch, train_loss, train_acc, valid_loss, valid_acc))
            if self.use_scheduler:
                self.scheduler.step(valid_loss)
            if valid_loss < self.best_loss:
                self.best_loss = valid_loss
                self.save_weights()
        return {"train_loss":train_losses, "train_accuracy":train_accuracies, "valid_loss":valid_losses, "valid_accuracy":valid_accuracies}

    def save_weights(self, ):
        torch.save(self.model.state_dict(), os.path.join(self.config.rootdir,"Models","BERT_NER","BERTF_NER.pth"))
        print("weights saved!!")
    
    def load_weights(self, dir):
        self.model.load_state_dict(torch.load(dir, map_location=self.device))
        print("weights loaded!!")

    def predict(self, sentence):
        self.model.eval()
        predictions = []
        inputs = []
        text = self.tokenizer(sentence, add_special_tokens=False, return_tensors="pt")
        text_ = self.tokenizer(sentence, add_special_tokens=False)
        input_id_chunks = text['input_ids'][0].split(self.config.maxlen-2)
        mask_chunks = text['attention_mask'][0].split(self.config.maxlen-2)
        for j in range(len(input_id_chunks)):
            inp_chunk = torch.cat([torch.Tensor([101]),input_id_chunks[j],torch.Tensor([102])])
            msk_chunk = torch.cat([torch.Tensor([1]), mask_chunks[j], torch.Tensor([1])])
            pad_len = self.config.maxlen - inp_chunk.shape[0]
            if pad_len > 0:
                inp_chunk = torch.cat([inp_chunk, torch.Tensor([0] * pad_len)])
                msk_chunk = torch.cat([msk_chunk, torch.Tensor([0] * pad_len)])
            input_dict = {
                          'input_ids': inp_chunk.long(),
                          'attention_mask': msk_chunk.int()
                         }
            inputs.append(input_dict)
        
        for inp in inputs:
            mask = inp['attention_mask'].unsqueeze(0).to(self.device)
            input_id = inp['input_ids'].unsqueeze(0).to(self.device)
            logits = self.model(input_id, mask, None).logits
            predictions.append(logits.argmax(dim=2).tolist()[0])

        split_tokens = sentence.split()
        output_words = []
        output_entities = []
        for p in range(len(predictions)):
            output_entities.extend(predictions[p][1:-1])
        comp_idx = 0
        extension = 0
        word_ids = text_.word_ids()
        label_ids = []
        i = 0
        while i < len(word_ids):
            if word_ids[i] is not None:
                word = self.tokenizer.decode(text_.input_ids[i-extension:i+1])
                if (word == split_tokens[comp_idx]) or ("".join(word.split()) == split_tokens[comp_idx]):
                    label_ids.append(self.config.id_tag[output_entities[i]])
                    comp_idx += 1
                    extension = 0
                elif word == '[UNK]':
                    label_ids.append(self.config.id_tag[output_entities[i]])
                    comp_idx += 1
                    extension = 0
                else:
                    extension += 1
            i += 1
        entities = {}
        for i in self.config.tag_id:
            entities[i] = []
        k = 0
        prev_label = None
        while k < len(label_ids):
            curr_label = label_ids[k]
            if curr_label != prev_label:
                entities[label_ids[k]].append(split_tokens[k])
            else:
                entities[label_ids[k]][-1] = entities[label_ids[k]][-1] + " " + split_tokens[k]
            prev_label = curr_label
            k += 1
        return entities

In [ ]:
val_text = text[:20]
val_labels = labels[:20]
train_text = text[20:]
train_labels = labels[20:]

In [ ]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=3,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):

        ce_loss = F.cross_entropy(input, target,reduction=self.reduction,weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self, config):
        super(BERTClass, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(config.tag_id))
    
    def forward(self, ids, mask, labels = None):
        output = self.bert(ids, mask, labels = labels)
        return output

In [ ]:
config = NERconfig(maxlen = 512, batch_size = 16, rootdir = ROOT_DIR, tag_id = tags_id, pretrained = True, use_scheduler = True)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
train_dataset = NER_Dataset(train_text, train_labels, config,  tokenizer) 
val_dataset = NER_Dataset(val_text, val_labels, config,  tokenizer)
criterion = FocalLoss()#weight = torch.tensor(weights)
BERTmodel = BERTClass(config)
optimizer = Adam(BERTmodel.parameters(), lr = 0.0001)

Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertF

In [ ]:
trainer = Trainer(
    BERTmodel,
    config,
    optimizer,
    tokenizer,
    criterion,
    train_dataset,
    val_dataset,
    val_dataset
)

Pretrained Weights found. Loading Pretrained weights...
weights loaded!!


In [ ]:
history = trainer.fit(100)

100%|██████████| 2/2 [00:00<00:00,  2.81it/s]


epoch 0 | train loss : 54.37728623472727 | train acc : 0.8611591045673077 | valid loss : 11.397019058465958 | valid acc : 0.91156005859375
weights saved!!


100%|██████████| 2/2 [00:00<00:00,  2.70it/s]


epoch 1 | train loss : 8.081614274245043 | train acc : 0.8809157151442307 | valid loss : 5.56871685385704 | valid acc : 0.87823486328125
weights saved!!


100%|██████████| 2/2 [00:00<00:00,  2.87it/s]


...

epoch 97 | train loss : 0.31371256938347447 | train acc : 0.8918832632211539 | valid loss : 12.468726396560669 | valid acc : 0.84417724609375


100%|██████████| 2/2 [00:00<00:00,  2.85it/s]


epoch 98 | train loss : 0.32345101781762564 | train acc : 0.891357421875 | valid loss : 3.766976833343506 | valid acc : 0.8582763671875


100%|██████████| 2/2 [00:00<00:00,  2.87it/s]

epoch 99 | train loss : 0.37331460473629147 | train acc : 0.8862116887019231 | valid loss : 7.509308695793152 | valid acc : 0.82220458984375


In [ ]:
pred = trainer.predict(text[5])

In [ ]:
for i in pred:
    if i != "Empty":
      for j in pred[i]:
          print('{:<20}{:<}'.format(i,j))

Name                Anvitha Rao
College Name        State
College Name        S Ramaiah
College Name        Technology
Degree              Masters in Computer
Degree              University
Degree              Bachelor of Engineering in Computer
Degree              M
Degree              Institute of
Companies worked at SAP Labs
Companies worked at SAP
Companies worked at SAP
Companies worked at SAP Labs
Designation         Automation developer
Skills              JAVA (1 year), C++ (Less than 1 year), Hadoop (Less than 1 year), HADOOP (Less than 1 year), CSS (Less than 1
Skills              Skills: Programming Languages: C, C++, HTML/CSS, Java, Python, Javascript Technologies: IoT, MySQL, PostgreSQL, D3js, Hadoop and Spark, Gephi
